In [ ]:
# モジュールのimport
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdate
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None) # pandasオプション：列データを全て表示

# データフレームdfにOur World in Dataのデータを読み込む
df = pd.read_json("https://covid.ourworldindata.org/data/owid-covid-data.json")
df = df[df.columns.drop(list(df.filter(regex='OWID')))] # OWIDデータを除外

countries = df.columns # 列の国名コード
date_keys = pd.DataFrame(df['USA']['data']).columns # dataに含まれるcolumn名（アメリカより取得）

df2 = pd.DataFrame(columns = countries, index = date_keys) # index：感染情報，column：国名コードとなるdf2を作成する

for column in countries:
    for indexs in date_keys:
        try:
            df2[column][indexs] = df[column]['data'][-7][indexs] # 1週間前のデータがあれば格納
        except LookupError:
            df2[column][indexs] = 'NaN' # なければNaN（not-a-number）を格納

# 型変換：objectをfloatへと変換
dft  = df.T.apply(pd.to_numeric, errors='ignore')
dft2 = df2.T.apply(pd.to_numeric, errors='coerce')

# 二つのデータフレームをマージ
df_all = pd.concat([dft,dft2], axis=1)

df_all # df_allの表示

In [ ]:
df_all.info() # テーブルの概要を確認

# 各columnの詳細情報は以下を確認すること
# URL: https://github.com/owid/covid-19-data/tree/master/public/data

In [ ]:
new = df_all[['population_density', 'median_age', 'gdp_per_capita', 'total_cases_per_million', 'total_deaths_per_million']].dropna(how='any')

new = new.apply(lambda x: (x-x.mean())/x.std(), axis=0)
new = new.drop(['SGP', 'CHN'])

new

In [ ]:
pd.plotting.scatter_matrix(new, figsize=(8, 8), alpha=0.5)
plt.show()

In [ ]:
pca = PCA()
pca.fit(new)

# 固有ベクトル
print(pca.components_)

# 固有値
print(pca.explained_variance_)

# 寄与率
print(pca.explained_variance_ratio_)

In [ ]:
reduced = pca.transform(new)
columns = [f'PC{i+1}' for i in range(len(new.columns))]
tdf = pd.DataFrame(data=reduced, index=new.index, columns=columns)
country_info = df_all[df_all.index.isin(new.index)][['continent', 'location']]

new2 = pd.concat([country_info, new,tdf], axis=1)
grouped = new2.groupby('continent')

new2

In [ ]:
cum_con_ratio = np.hstack([0, pca.explained_variance_ratio_]).cumsum()
plt.plot(cum_con_ratio, '-o')
plt.show()

In [ ]:
# 図のテンプレートを準備
fig, ax = plt.subplots()

# 横軸・縦軸の項目指定
key1 = 'PC1'
key2 = 'PC2'

# 色指定：Matpolotlibの標準色を採用; 辞書型
colors = {'Africa':'#1f77b4', 'Asia':'#ff7f0e', 'Europe':'#2ca02c', 'North America':'#d62728', 'Oceania':'#9467bd', 'South America':'#8c564b'}

# for文により大陸毎の散布図をループ
for key, group in grouped:
    group.plot.scatter(x=key1, y=key2, ax=ax, label=key, color=colors[key])

for index, row in new2.iterrows():
    ax.annotate(index, (row[key1], row[key2])) # 座標(row[key1], row[key2])の位置に国名コードであるindexを表示

#ax.set_xlim([-2.0, 0.0])
#ax.set_ylim([-1.0, 1.0])

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.quiver(np.zeros(5)+1.0e-7, np.zeros(5)+1.0e-7, pca.components_[0], pca.components_[1], angles='xy',scale=1, scale_units='xy', color='red')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])

for x, y, name in zip(pca.components_[0], pca.components_[1], new.columns[0:]):
    plt.text(x, y, name)

ax.set_xlabel('PC1')
ax.set_xlabel('PC2')
plt.show()